In [1]:
import pandas as pd
from tqdm import tqdm
from itertools import combinations

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
X = pd.read_parquet('df_X.parquet.gzip') 
df_subm = pd.read_csv('sample_submission.csv', index_col='id')
X_test = pd.read_parquet('df_test.parquet.gzip') 

In [2]:
y = X['Listening_Time_minutes']
TE_col = X.columns[207:]
TE_col

Index(['Episode_Length_minutes_Episode_Title',
       'Episode_Length_minutes_Host_Popularity_percentage',
       'Episode_Length_minutes_Number_of_Ads',
       'Episode_Length_minutes_Episode_Sentiment',
       'Episode_Length_minutes_Publication_Day',
       'Episode_Length_minutes_Publication_Time',
       'Episode_Length_minutes_Podcast_Name',
       'Episode_Length_minutes_Guest_Popularity_percentage',
       'Episode_Length_minutes_Genre',
       'Episode_Title_Host_Popularity_percentage',
       ...
       'Episode_Sentiment_Publication_Day_Guest_Popularity_percentage_Genre',
       'Episode_Sentiment_Publication_Time_Podcast_Name_Guest_Popularity_percentage',
       'Episode_Sentiment_Publication_Time_Podcast_Name_Genre',
       'Episode_Sentiment_Publication_Time_Guest_Popularity_percentage_Genre',
       'Episode_Sentiment_Podcast_Name_Guest_Popularity_percentage_Genre',
       'Publication_Day_Publication_Time_Podcast_Name_Guest_Popularity_percentage',
       'Publication_Da

In [3]:
cv = KFold(5, random_state=42, shuffle=True)
idx = list(cv.split(X, y))
len(idx[0][1])

158974

In [4]:
encode_columns = ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Number_of_Ads'
                  ,'Episode_Sentiment', 'Publication_Day', 'Publication_Time','Podcast_Name','Guest_Popularity_percentage'
                 ,'Genre']
TE2 = []
for cols in tqdm(list(combinations(encode_columns, 2))):
        new_col_name = '_'.join(cols)
        TE2.append(new_col_name)

encode_columns = ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Number_of_Ads'
                  ,'Episode_Sentiment', 'Publication_Day', 'Publication_Time','Podcast_Name','Guest_Popularity_percentage'
                 ,'Genre']
TE3 = []
for cols in tqdm(list(combinations(encode_columns, 3))):
        new_col_name = '_'.join(cols)
        TE3.append(new_col_name)

100%|████████████████████████████████████| 120/120 [00:00<00:00, 3050402.91it/s]


In [5]:
def Feature_eng(idx_train,idx_valid):
    global X_test, X
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    for x in [0.05,0.10,0.40,0.45,0.55,0.60,0.90,0.95]:
        for w in ['Listening_Time_minutes']:
            for z in ['Publication_Day','Podcast_Name','Publication_Time','Episode_Title','Genre','NaNs'
                      , 'Episode_Length_minutes_round0','Host_Popularity_percentage_round0','Guest_Popularity_percentage_round0'
                      , 'Episode_Length_minutes_digit0','Episode_Length_minutes_digit1', 'Host_Popularity_percentage_digit0'
                      ,'Host_Popularity_percentage_digit1','Guest_Popularity_percentage_digit0','Guest_Popularity_percentage_digit1']:
                name = w + str(x) + z
                a = X_train.groupby(z)[w].quantile(x).reset_index(name=name)
                X_train = X_train.merge(a,on=z,how='left')
                X_test = X_test.merge(a,on=z,how='left')      
                X_valid = X_valid.merge(a,on=z,how='left') 
    print(X_valid.shape,len(y_valid))
    print('2')

    for x in ['mean','max','min','std','quantile','skew','sum']:
        for w in ['Episode_Length_minutes']:
            for z in ['Publication_Day','Podcast_Name','Publication_Time','Episode_Title','Genre','Number_of_Ads']:
                name = w + x + z
                a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
                X_train = X_train.merge(a,on=z,how='left')
                X_test = X_test.merge(a,on=z,how='left')      
                X_valid = X_valid.merge(a,on=z,how='left') 
    print(X_valid.shape,len(y_valid))
    print(X_train.shape,len(y_valid))
    print('1')

    for x in [5,10,15,20,25,50,100,150]:
        name = 'Episode_Length_minutes' + str(x)
        X_train["Episode_Length_minutes_copy"] = X_train["Episode_Length_minutes"].copy()
        X_valid["Episode_Length_minutes_copy"] = X_valid["Episode_Length_minutes"].copy()
        X_test["Episode_Length_minutes_copy"] = X_test["Episode_Length_minutes"].copy()
        
        ser, bins = pd.qcut(X_train["Episode_Length_minutes_copy"], x, retbins=True, labels=False)
        X_train[name] = pd.cut(X_train['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
        X_valid[name] = pd.cut(X_valid['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
        X_test[name] = pd.cut(X_test['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    
    for x in ['mean','quantile']:
        for w in ['Listening_Time_minutes']:
            for z in ['Episode_Length_minutes5','Episode_Length_minutes10','Episode_Length_minutes15','Episode_Length_minutes20'
                     ,'Episode_Length_minutes25','Episode_Length_minutes50','Episode_Length_minutes100','Episode_Length_minutes150']:
                name = w + x + z
                a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
                X_train = X_train.merge(a,on=z,how='left')
                X_test = X_test.merge(a,on=z,how='left')      
                X_valid = X_valid.merge(a,on=z,how='left') 
    print(X_valid.shape,len(y_valid))
    print('8')

    for x in ['count']:
        for w in ['Publication_Day','Podcast_Name','Publication_Time','Episode_Title','Genre']:
            name = w + x
            a = X_train.reset_index().groupby(w)['index'].apply(x).reset_index(name=name)
            X_train = X_train.merge(a,on=w)
            X_test = X_test.merge(a,on=w)
            X_valid = X_valid.merge(a,on=w)   
    print(X_valid.shape,len(y_valid))
    print('5')
    
    print('6')
    return X_train,X_valid,y_train,y_valid


In [ ]:
#in use
def Feature_eng(idx_train,idx_valid):
    global X_test, X
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    for x in ['mean','max','min','std','quantile','skew','sum']:
        for w in ['Listening_Time_minutes']:
            for z in ['NaNs', 'Episode_Length_minutes_round0','Host_Popularity_percentage_round0','Guest_Popularity_percentage_round0'
                      , 'Episode_Length_minutes_digit0','Episode_Length_minutes_digit1', 'Host_Popularity_percentage_digit0'
                      ,'Host_Popularity_percentage_digit1','Guest_Popularity_percentage_digit0','Guest_Popularity_percentage_digit1'
                     ,'Publication_Day','Podcast_Name','Publication_Time','Episode_Title','Genre','Number_of_Ads']:
                name = w + x + z
                a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
                X_train = X_train.merge(a,on=z,how='left')
                X_test = X_test.merge(a,on=z,how='left')      
                X_valid = X_valid.merge(a,on=z,how='left')
    print(X_valid.shape,len(y_valid))

    encoded_columns = TE_col
    encoder = TargetEncoder(random_state=42)
    
    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])
    print(X_valid.shape,len(y_valid))

    for x in TE2:
        a = 'Episode_Length_minutes' + '_div_' + x
        X_train[a] = X_train['Episode_Length_minutes'] / X_train[x]
        X_valid[a] = X_valid['Episode_Length_minutes'] / X_valid[x]
        X_test[a] = X_test['Episode_Length_minutes'] / X_test[x]
    print(X_valid.shape,len(y_valid))
    
    for x in TE3:
        a = 'Episode_Length_minutes' + '_div_' + x
        X_train[a] = X_train['Episode_Length_minutes'] / X_train[x]
        X_valid[a] = X_valid['Episode_Length_minutes'] / X_valid[x]
        X_test[a] = X_test['Episode_Length_minutes'] / X_test[x]
    print(X_valid.shape,len(y_valid))
    
    X_train['SinEpLen'] = np.sin(2*np.pi * X_train['Episode_Length_minutes']/60 )
    X_train['CosEpLen'] = np.cos(2*np.pi * X_train['Episode_Length_minutes']/60 )
    X_train["ELen_Int"] = np.floor(X_train["Episode_Length_minutes"])
    X_train["ELen_Dec"] = X_train["Episode_Length_minutes"] - X_train["ELen_Int"]
    
    X_test['SinEpLen'] = np.sin(2*np.pi * X_test['Episode_Length_minutes']/60 )
    X_test['CosEpLen'] = np.cos(2*np.pi * X_test['Episode_Length_minutes']/60 )
    X_test["ELen_Int"] = np.floor(X_test["Episode_Length_minutes"])
    X_test["ELen_Dec"] = X_test["Episode_Length_minutes"] - X_test["ELen_Int"]
    
    X_valid['SinEpLen'] = np.sin(2*np.pi * X_valid['Episode_Length_minutes']/60 )
    X_valid['CosEpLen'] = np.cos(2*np.pi * X_valid['Episode_Length_minutes']/60 )
    X_valid["ELen_Int"] = np.floor(X_valid["Episode_Length_minutes"])
    X_valid["ELen_Dec"] = X_valid["Episode_Length_minutes"] - X_valid["ELen_Int"]
    print(X_valid.shape,len(y_valid))
    return X_train,X_valid,y_train,y_valid


In [6]:
idx_train = idx[0][0]
idx_valid = idx[0][1]

X_train,X_valid,y_train,y_valid = Feature_eng(idx_train,idx_valid)

X_train.to_parquet('X_train_1.parquet.gzip',
          compression='gzip') 
X_valid.to_parquet('X_valid_1.parquet.gzip',
          compression='gzip') 
X_test.to_parquet('X_test_1.parquet.gzip',
          compression='gzip') 
y_train.to_csv('y_train_1.csv') 
y_valid.to_csv('y_valid_1.csv') 
print('7')

(158974, 694) 158974
(158974, 694) 158974
(158974, 739) 158974
(158974, 859) 158974
(158974, 863) 158974
7


In [6]:
idx_train = idx[1][0]
idx_valid = idx[1][1]

X_train,X_valid,y_train,y_valid = Feature_eng(idx_train,idx_valid)

X_train.to_parquet('X_train_2.parquet.gzip',
          compression='gzip') 
X_valid.to_parquet('X_valid_2.parquet.gzip',
          compression='gzip') 
X_test.to_parquet('X_test_2.parquet.gzip',
          compression='gzip') 
y_train.to_csv('y_train_2.csv') 
y_valid.to_csv('y_valid_2.csv') 
print('7')

(158974, 636) 158974
8
(158974, 671) 158974
1
(158974, 791) 158974
2
(158974, 833) 158974
(635893, 833) 158974
1
(158974, 838) 158974
5
6
7


In [6]:
idx_train = idx[2][0]
idx_valid = idx[2][1]

X_train,X_valid,y_train,y_valid = Feature_eng(idx_train,idx_valid)

X_train.to_parquet('X_train_3.parquet.gzip',
          compression='gzip') 
X_valid.to_parquet('X_valid_3.parquet.gzip',
          compression='gzip') 
X_test.to_parquet('X_test_3.parquet.gzip',
          compression='gzip') 
y_train.to_csv('y_train_3.csv') 
y_valid.to_csv('y_valid_3.csv') 
print('7')

(158973, 636) 158973
8
(158973, 671) 158973
1
(158973, 791) 158973
2
(158973, 833) 158973
(635894, 833) 158973
1
(158973, 838) 158973
5
6
7


In [6]:
idx_train = idx[3][0]
idx_valid = idx[3][1]

X_train,X_valid,y_train,y_valid = Feature_eng(idx_train,idx_valid)

X_train.to_parquet('X_train_4.parquet.gzip',
          compression='gzip') 
X_valid.to_parquet('X_valid_4.parquet.gzip',
          compression='gzip') 
X_test.to_parquet('X_test_4.parquet.gzip',
          compression='gzip') 
y_train.to_csv('y_train_4.csv') 
y_valid.to_csv('y_valid_4.csv') 
print('7')

(158973, 636) 158973
8
(158973, 671) 158973
1
(158973, 791) 158973
2
(158973, 833) 158973
(635894, 833) 158973
1
(158973, 838) 158973
5
6
7


In [6]:
idx_train = idx[4][0]
idx_valid = idx[4][1]

X_train,X_valid,y_train,y_valid = Feature_eng(idx_train,idx_valid)

X_train.to_parquet('X_train_5.parquet.gzip',
          compression='gzip') 
X_valid.to_parquet('X_valid_5.parquet.gzip',
          compression='gzip') 
X_test.to_parquet('X_test_5.parquet.gzip',
          compression='gzip') 
y_train.to_csv('y_train_5.csv') 
y_valid.to_csv('y_valid_5.csv') 
print('7')

(158973, 636) 158973
8
(158973, 671) 158973
1
(158973, 791) 158973
2
(158973, 833) 158973
(635894, 833) 158973
1
(158973, 838) 158973
5
6
7
